<a href="https://colab.research.google.com/github/Malanidhruv/Python-projects/blob/main/Stocks%20Research/Trades%20for%20tomorrow/SWING%20TRADES.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
%pip install pya3
%pip install nsetools

## Including todays data

In [8]:
import numpy as np
import pandas as pd
from scipy.signal import argrelextrema
from pya3 import Aliceblue
from datetime import datetime, time
from nsetools import Nse

def get_user_credentials():
    user_id = input("Enter your user ID: ")
    api_key = input("Enter your API key: ")
    return user_id, api_key

def initialize_alice():
    user_id, api_key = get_user_credentials()
    alice = Aliceblue(user_id=user_id, api_key=api_key)
    alice.get_session_id()
    return alice

def get_today_data(alice, token):
    current_date = datetime.now()
    instrument = alice.get_instrument_by_token('NSE', token)

    from_datetime = datetime.combine(current_date, time(9, 15))
    to_datetime = datetime.combine(current_date, time(15, 30))
    interval = "1"  # Minute-wise data

    historical_data = alice.get_historical(instrument, from_datetime, to_datetime, interval)

    # Convert the list of dictionaries to a DataFrame
    df_today = pd.DataFrame(historical_data)

    # Check if 'datetime' key exists
    if 'datetime' in df_today.columns:
        df_today['datetime'] = pd.to_datetime(df_today['datetime'])
    else:
        raise KeyError("'datetime' key not found in historical data")

    start_time = datetime.combine(current_date, time(9, 15))
    end_time = datetime.combine(current_date, time(15, 15, 59))
    filtered_data = df_today[(df_today['datetime'] >= start_time) &
                             (df_today['datetime'] <= end_time)]

    cumulative_volume = filtered_data['volume'].sum()

    new_daily_data = {
        'datetime': [end_time],
        'open': [filtered_data.iloc[0]['open']],
        'high': [filtered_data['high'].max()],
        'low': [filtered_data['low'].min()],
        'close': [filtered_data.iloc[-1]['close']],
        'volume': [cumulative_volume]
    }
    return pd.DataFrame(new_daily_data)

def get_historical_data(alice, instrument):
    from_datetime = datetime(2022, 1, 1)
    to_datetime = datetime.now()
    interval = "D"
    historical_data = alice.get_historical(instrument, from_datetime, to_datetime, interval)
    return pd.DataFrame(historical_data)

def check_near_extremes(current_price, close_prices, local_minima_indices, threshold=0.05):
    def is_near_extreme(current_price, extreme_price, threshold):
        return abs((current_price - extreme_price) / extreme_price) < threshold

    near_local_minima = any(is_near_extreme(current_price, close_prices[i], threshold) for i in local_minima_indices)
    global_minima_index = np.argmin(close_prices)
    near_global_minima = is_near_extreme(current_price, close_prices[global_minima_index], threshold)

    return near_local_minima, near_global_minima, global_minima_index

def should_buy_signal(near_local_minima, near_global_minima, current_price, close_prices, local_minima_indices):
    if (near_local_minima or near_global_minima):
        if near_local_minima:
            nearest_local_minima_index = local_minima_indices[np.argmin(abs(close_prices[local_minima_indices] - current_price))]
            return current_price > close_prices[nearest_local_minima_index]
        else:
            global_minima_index = np.argmin(close_prices)
            return current_price > close_prices[global_minima_index]
    return False

def get_support_level_and_volume(df, close_prices, local_minima_indices, near_local_minima):
    if near_local_minima:
        nearest_local_minima_index = local_minima_indices[np.argmin(abs(close_prices[local_minima_indices] - df['close'].iloc[-1]))]
        support_level = close_prices[nearest_local_minima_index]
        support_volume = df.loc[nearest_local_minima_index, 'volume']
    else:
        global_minima_index = np.argmin(close_prices)
        support_level = close_prices[global_minima_index]
        support_volume = df.loc[global_minima_index, 'volume']

    return support_level, support_volume

def analyze_stock(alice, token):
    instrument = alice.get_instrument_by_token('NSE', token)
    historical_data = get_historical_data(alice, instrument)
    today_data = get_today_data(alice, token)
    df = pd.concat([historical_data, today_data], ignore_index=True)

    close_prices = df['close'].values
    local_maxima_indices = argrelextrema(close_prices, np.greater)[0]
    local_minima_indices = argrelextrema(close_prices, np.less)[0]
    current_price = df['close'].iloc[-1]

    near_local_minima, near_global_minima, global_minima_index = check_near_extremes(current_price, close_prices, local_minima_indices)

    if should_buy_signal(near_local_minima, near_global_minima, current_price, close_prices, local_minima_indices):
        support_level, support_volume = get_support_level_and_volume(df, close_prices, local_minima_indices, near_local_minima)
        volume_threshold = 1.5 * df['volume'].mean()
        if support_volume < volume_threshold:
            return token, current_price, None, None
        return token, current_price, "BUY", support_level

    return token, current_price, None, None

def print_signals(buy_signals):
    print("BUY Signals:")
    print(f"{'Name':<15} {'Token':<10} {'Current Price':<15} {'Support Level':<15}")
    print('-' * 55)
    for signal in buy_signals:
        print(f"{signal['Name']:<15} {signal['Token']:<10} {signal['Current Price']:<15.2f} {signal['Support Level']:<15.2f}")
    print("--------------------------------------------------")

def get_stock_name(alice, token):
    instrument = alice.get_instrument_by_token('NSE', token)
    return instrument.name

# Initialize Aliceblue object and get credentials
alice = initialize_alice()

# List of tokens to analyze
tokens = [
    11532, 4306, 467, 2885, 1394, 10999, 547, 1232, 1922, 526, 17818, 21808, 17963, 1660, 11536, 15083, 3456, 11483, 13538, 236, 20374, 10604, 25, 14977, 4963, 5258, 1594, 10940, 3351, 3787, 11630, 3432, 317, 7229, 3045, 1348, 16675, 881, 3506, 11723, 2475, 1333, 910, 5900, 694, 2031, 157, 1363, 16669, 3499
]

# Analyze each stock
buy_signals = []

for token in tokens:
    try:
        result = analyze_stock(alice, token)
        if result[2] == "BUY":
            buy_signals.append((result[0], result[1], result[3]))
    except Exception as e:
        print(f"Error analyzing token {token}: {e}")

# Adding stock names to the signals
for i in range(len(buy_signals)):
    buy_signals[i] = {
        'Token': buy_signals[i][0],
        'Current Price': buy_signals[i][1],
        'Support Level': buy_signals[i][2],
        'Name': get_stock_name(alice, buy_signals[i][0])
    }

# Print signals
print_signals(buy_signals)

Enter your user ID: 1141826
Enter your API key: uGmlgEfgvtLmlSfnPUvZJDuOfLOU6r0JVWlWpJezmeBBlGZuf63c6x1hBBmFdI7BcE15pQzpucaaNNJ4e8COP4LXqOSsSvLfMEh4qB6ZligOWjq5869N7hPO1TnvnoZ4
Error analyzing token 2031: If using all scalar values, you must pass an index
BUY Signals:
Name            Token      Current Price   Support Level  
-------------------------------------------------------
RELIANCE-EQ     2885       1310.00         1302.15        
KOTAKBANK-EQ    1922       1757.75         1757.55        
TECHM-EQ        13538      1760.85         1732.60        
WIPRO-EQ        3787       294.00          291.65         
HCLTECH-EQ      7229       1899.60         1891.70        
DRREDDY-EQ      881        1214.85         1195.35        
--------------------------------------------------


# Doesn't includes todays data

In [9]:
import numpy as np
import pandas as pd
from scipy.signal import argrelextrema
from pya3 import Aliceblue
from datetime import datetime

def get_user_credentials():
    user_id = input("Enter your user ID: ")
    api_key = input("Enter your API key: ")
    return user_id, api_key

def initialize_alice():
    user_id, api_key = get_user_credentials()
    alice = Aliceblue(user_id=user_id, api_key=api_key)
    alice.get_session_id()
    return alice

def analyze_stock(alice, token):
    try:
        instrument = alice.get_instrument_by_token('NSE', token)
        from_datetime = datetime(2022, 1, 1)
        to_datetime = datetime.now()
        interval = "D"

        historical_data = alice.get_historical(instrument, from_datetime, to_datetime, interval)
        df = pd.DataFrame(historical_data)

        if df.empty:
            print(f"No historical data for token {token}.")
            return token, None, None, None

        close_prices = df['close'].values
        local_maxima_indices = argrelextrema(close_prices, np.greater)[0]
        local_minima_indices = argrelextrema(close_prices, np.less)[0]

        global_maxima_index = np.argmax(close_prices)
        global_minima_index = np.argmin(close_prices)

        current_price = df['close'].iloc[-1]
        threshold = 0.05

        def is_near_extreme(current_price, extreme_price, threshold):
            return abs((current_price - extreme_price) / extreme_price) < threshold

        near_local_minima = any(is_near_extreme(current_price, close_prices[i], threshold) for i in local_minima_indices)
        near_global_minima = is_near_extreme(current_price, close_prices[global_minima_index], threshold)

        df['volume_ma'] = df['volume'].rolling(window=20).mean()
        volume_threshold = 1.5 * df['volume'].mean()

        if (near_local_minima or near_global_minima) and (current_price > (close_prices[local_minima_indices[np.argmin(abs(close_prices[local_minima_indices] - current_price))]] if near_local_minima else close_prices[global_minima_index])):
            signal = "BUY"
            if near_local_minima:
                nearest_local_minima_index = local_minima_indices[np.argmin(abs(close_prices[local_minima_indices] - current_price))]
                support_level = close_prices[nearest_local_minima_index]
                support_volume = df.loc[nearest_local_minima_index, 'volume']
            else:
                support_level = close_prices[global_minima_index]
                support_volume = df.loc[global_minima_index, 'volume']
            if support_volume < volume_threshold:
                signal = None
        else:
            signal = None

        return token, current_price, signal, support_level if signal == "BUY" else None

    except Exception as e:
        print(f"Error analyzing token {token}: {e}")
        return token, None, None, None

def print_signals(buy_signals):
    print("BUY Signals:")
    print(f"{'Name':<15} {'Token':<10} {'Current Price':<15} {'Support Level':<15}")
    print('-' * 55)
    for signal in buy_signals:
        print(f"{signal['Name']:<15} {signal['Token']:<10} {signal['Current Price']:<15.2f} {signal['Support Level']:<15.2f}")
    print("--------------------------------------------------")

def get_stock_name(alice, token):
    instrument = alice.get_instrument_by_token('NSE', token)
    return instrument.name

# Initialize Aliceblue object and get credentials
alice = initialize_alice()

# List of tokens to analyze
tokens = [
    11532, 4306, 467, 2885, 1394, 10999, 547, 1232, 1922, 526, 17818, 21808, 17963, 1660, 11536, 15083, 3456, 11483, 13538, 236, 20374, 10604, 25, 14977, 4963, 5258, 1594, 10940, 3351, 3787, 11630, 3432, 317, 7229, 3045, 1348, 16675, 881, 3506, 11723, 2475, 1333, 910, 5900, 694, 2031, 157, 1363, 16669, 3499
]

# Analyze each stock
buy_signals = []

for token in tokens:
    result = analyze_stock(alice, token)
    if result[2] == "BUY":
        buy_signals.append((result[0], result[1], result[3]))

# Adding stock names to the signals
for i in range(len(buy_signals)):
    buy_signals[i] = {
        'Token': buy_signals[i][0],
        'Current Price': buy_signals[i][1],
        'Support Level': buy_signals[i][2],
        'Name': get_stock_name(alice, buy_signals[i][0])
    }

# Print signals
print_signals(buy_signals)

Enter your user ID: 1141826
Enter your API key: uGmlgEfgvtLmlSfnPUvZJDuOfLOU6r0JVWlWpJezmeBBlGZuf63c6x1hBBmFdI7BcE15pQzpucaaNNJ4e8COP4LXqOSsSvLfMEh4qB6ZligOWjq5869N7hPO1TnvnoZ4
BUY Signals:
Name            Token      Current Price   Support Level  
-------------------------------------------------------
TECHM-EQ        13538      1749.50         1732.60        
DIVISLAB-EQ     10940      6210.55         6106.70        
HEROMOTOCO-EQ   1348       4697.00         4687.50        
BAJAJFINSV-EQ   16675      1607.10         1606.70        
BAJAJ-AUTO-EQ   16669      9161.80         9137.45        
TATASTEEL-EQ    3499       146.54          146.17         
--------------------------------------------------


In [12]:
import numpy as np
import pandas as pd
from scipy.signal import argrelextrema
from pya3 import Aliceblue
from datetime import datetime

def get_user_credentials():
    user_id = input("Enter your user ID: ")
    api_key = input("Enter your API key: ")
    return user_id, api_key

def initialize_alice():
    user_id, api_key = get_user_credentials()
    alice = Aliceblue(user_id=user_id, api_key=api_key)
    alice.get_session_id()
    return alice

def analyze_stock(alice, token):
    try:
        instrument = alice.get_instrument_by_token('NSE', token)
        from_datetime = datetime(2022, 1, 1)
        to_datetime = datetime.now()
        interval = "D"

        historical_data = alice.get_historical(instrument, from_datetime, to_datetime, interval)
        df = pd.DataFrame(historical_data)

        if df.empty:
            print(f"No historical data for token {token}.")
            return token, None, None, None

        close_prices = df['close'].values
        local_maxima_indices = argrelextrema(close_prices, np.greater)[0]
        local_minima_indices = argrelextrema(close_prices, np.less)[0]

        global_maxima_index = np.argmax(close_prices)
        global_minima_index = np.argmin(close_prices)

        current_price = df['close'].iloc[-1]
        threshold = 0.05

        def is_near_extreme(current_price, extreme_price, threshold):
            return abs((current_price - extreme_price) / extreme_price) < threshold

        near_local_minima = any(is_near_extreme(current_price, close_prices[i], threshold) for i in local_minima_indices)
        near_global_minima = is_near_extreme(current_price, close_prices[global_minima_index], threshold)

        df['volume_ma'] = df['volume'].rolling(window=20).mean()
        volume_threshold = 1.5 * df['volume'].mean()

        if (near_local_minima or near_global_minima) and (current_price > (close_prices[local_minima_indices[np.argmin(abs(close_prices[local_minima_indices] - current_price))]] if near_local_minima else close_prices[global_minima_index])):
            signal = "BUY"
            if near_local_minima:
                nearest_local_minima_index = local_minima_indices[np.argmin(abs(close_prices[local_minima_indices] - current_price))]
                support_level = close_prices[nearest_local_minima_index]
                support_volume = df.loc[nearest_local_minima_index, 'volume']
            else:
                support_level = close_prices[global_minima_index]
                support_volume = df.loc[global_minima_index, 'volume']
            if support_volume < volume_threshold:
                signal = None
        else:
            signal = None

        return token, current_price, signal, support_level if signal == "BUY" else None

    except Exception as e:
        print(f"Error analyzing token {token}: {e}")
        return token, None, None, None

def print_signals(buy_signals):
    print("BUY Signals:")
    print(f"{'Name':<15} {'Token':<10} {'Current Price':<15} {'Support Level':<15}")
    print('-' * 55)
    for signal in buy_signals:
        print(f"{signal['Name']:<15} {signal['Token']:<10} {signal['Current Price']:<15.2f} {signal['Support Level']:<15.2f}")
    print("--------------------------------------------------")

def get_stock_name(alice, token):
    instrument = alice.get_instrument_by_token('NSE', token)
    return instrument.name

# Initialize Aliceblue object and get credentials
alice = initialize_alice()

# List of tokens to analyze
tokens = [7,13,17903,21614,30108,22,25,15083,11703,1270,157,163,212,236,14418,263,21238,275,5900,16669,16675,317,335,341,2263,4668,
          371,383,404,422,10604,438,11373,2181,526,547,6994,10794,583,637,685,694,20374,11543,15141,4749,739,17094,5701,1901,772,8075,
          19943,10940,21690,14732,881,910,958,676,1023,4717,7406,13528,1174,10099,17875,11872,1232,10599,2303,9819,7229,1333,4244,1333,
          467,1348,1363,17939,1406,1394,4963,21770,18652,14366,11184,11184,220,11262,1512,1515,10726,11195,5258,29135,1594,1624,1633,
          13611,1660,6733,13270,11723,18096,1922,18564,11654,19234,1997,11483,24948,17818,10440,2031,13285,19061,4067,10999,31181,9581,
          2142,17534,4204,4503,2277,23650,6364,13751,14672,17963,15332,11630,20242,10738,2475,14413,2412,18365,11351,14299,2664,24184,
          10666,9590,14977,13147,2043,18391,15355,2885,2963,17971,21808,3045,3103,3150,3273,3351,13404,10243,3405,3721,3432,3456,3426,3499,
          11536,13538,3506,3518,10447,1964,8479,16713,11532,11287,3063,3718,3787,3812,7929
]
# Analyze each stock
buy_signals = []

for token in tokens:
    result = analyze_stock(alice, token)
    if result[2] == "BUY":
        buy_signals.append((result[0], result[1], result[3]))

# Adding stock names to the signals
for i in range(len(buy_signals)):
    buy_signals[i] = {
        'Token': buy_signals[i][0],
        'Current Price': buy_signals[i][1],
        'Support Level': buy_signals[i][2],
        'Name': get_stock_name(alice, buy_signals[i][0])
    }

# Print signals
print_signals(buy_signals)

Enter your user ID: 1141826
Enter your API key: uGmlgEfgvtLmlSfnPUvZJDuOfLOU6r0JVWlWpJezmeBBlGZuf63c6x1hBBmFdI7BcE15pQzpucaaNNJ4e8COP4LXqOSsSvLfMEh4qB6ZligOWjq5869N7hPO1TnvnoZ4
BUY Signals:
Name            Token      Current Price   Support Level  
-------------------------------------------------------
ABCAPITAL-EQ    21614      199.47          199.10         
BAJAJ-AUTO-EQ   16669      9161.80         9137.45        
BAJAJFINSV-EQ   16675      1607.10         1606.70        
BANDHANBNK-EQ   2263       175.27          173.80         
BEL-EQ          383        312.10          309.30         
BERGEPAINT-EQ   404        482.15          481.25         
BIOCON-EQ       11373      373.85          373.70         
CUMMINSIND-EQ   1901       3512.30         3497.90        
DIVISLAB-EQ     10940      6210.55         6106.70        
HEROMOTOCO-EQ   1348       4697.00         4687.50        
IOC-EQ          1624       139.51          138.95         
MANAPPURAM-EQ   19061      164.19          164

INCLUDING TODAYS DATA

In [14]:
import numpy as np
import pandas as pd
from scipy.signal import argrelextrema
from pya3 import Aliceblue
from datetime import datetime, time
from nsetools import Nse

def get_user_credentials():
    user_id = input("Enter your user ID: ")
    api_key = input("Enter your API key: ")
    return user_id, api_key

def initialize_alice():
    user_id, api_key = get_user_credentials()
    alice = Aliceblue(user_id=user_id, api_key=api_key)
    alice.get_session_id()
    return alice

def get_today_data(alice, token):
    current_date = datetime.now()
    instrument = alice.get_instrument_by_token('NSE', token)

    from_datetime = datetime.combine(current_date, time(9, 15))
    to_datetime = datetime.combine(current_date, time(15, 30))
    interval = "1"  # Minute-wise data

    historical_data = alice.get_historical(instrument, from_datetime, to_datetime, interval)

    # Convert the list of dictionaries to a DataFrame
    df_today = pd.DataFrame(historical_data)

    # Check if 'datetime' key exists
    if 'datetime' in df_today.columns:
        df_today['datetime'] = pd.to_datetime(df_today['datetime'])
    else:
        raise KeyError("'datetime' key not found in historical data")

    start_time = datetime.combine(current_date, time(9, 15))
    end_time = datetime.combine(current_date, time(15, 15, 59))
    filtered_data = df_today[(df_today['datetime'] >= start_time) &
                             (df_today['datetime'] <= end_time)]

    cumulative_volume = filtered_data['volume'].sum()

    new_daily_data = {
        'datetime': [end_time],
        'open': [filtered_data.iloc[0]['open']],
        'high': [filtered_data['high'].max()],
        'low': [filtered_data['low'].min()],
        'close': [filtered_data.iloc[-1]['close']],
        'volume': [cumulative_volume]
    }
    return pd.DataFrame(new_daily_data)

def get_historical_data(alice, instrument):
    from_datetime = datetime(2022, 1, 1)
    to_datetime = datetime.now()
    interval = "D"
    historical_data = alice.get_historical(instrument, from_datetime, to_datetime, interval)
    return pd.DataFrame(historical_data)

def check_near_extremes(current_price, close_prices, local_minima_indices, threshold=0.05):
    def is_near_extreme(current_price, extreme_price, threshold):
        return abs((current_price - extreme_price) / extreme_price) < threshold

    near_local_minima = any(is_near_extreme(current_price, close_prices[i], threshold) for i in local_minima_indices)
    global_minima_index = np.argmin(close_prices)
    near_global_minima = is_near_extreme(current_price, close_prices[global_minima_index], threshold)

    return near_local_minima, near_global_minima, global_minima_index

def should_buy_signal(near_local_minima, near_global_minima, current_price, close_prices, local_minima_indices):
    if (near_local_minima or near_global_minima):
        if near_local_minima:
            nearest_local_minima_index = local_minima_indices[np.argmin(abs(close_prices[local_minima_indices] - current_price))]
            return current_price > close_prices[nearest_local_minima_index]
        else:
            global_minima_index = np.argmin(close_prices)
            return current_price > close_prices[global_minima_index]
    return False

def get_support_level_and_volume(df, close_prices, local_minima_indices, near_local_minima):
    if near_local_minima:
        nearest_local_minima_index = local_minima_indices[np.argmin(abs(close_prices[local_minima_indices] - df['close'].iloc[-1]))]
        support_level = close_prices[nearest_local_minima_index]
        support_volume = df.loc[nearest_local_minima_index, 'volume']
    else:
        global_minima_index = np.argmin(close_prices)
        support_level = close_prices[global_minima_index]
        support_volume = df.loc[global_minima_index, 'volume']

    return support_level, support_volume

def analyze_stock(alice, token):
    instrument = alice.get_instrument_by_token('NSE', token)
    historical_data = get_historical_data(alice, instrument)
    today_data = get_today_data(alice, token)
    df = pd.concat([historical_data, today_data], ignore_index=True)

    close_prices = df['close'].values
    local_maxima_indices = argrelextrema(close_prices, np.greater)[0]
    local_minima_indices = argrelextrema(close_prices, np.less)[0]
    current_price = df['close'].iloc[-1]

    near_local_minima, near_global_minima, global_minima_index = check_near_extremes(current_price, close_prices, local_minima_indices)

    if should_buy_signal(near_local_minima, near_global_minima, current_price, close_prices, local_minima_indices):
        support_level, support_volume = get_support_level_and_volume(df, close_prices, local_minima_indices, near_local_minima)
        volume_threshold = 1.5 * df['volume'].mean()
        if support_volume < volume_threshold:
            return token, current_price, None, None
        return token, current_price, "BUY", support_level

    return token, current_price, None, None

def print_signals(buy_signals):
    print("BUY Signals:")
    print(f"{'Name':<15} {'Token':<10} {'Current Price':<15} {'Support Level':<15}")
    print('-' * 55)
    for signal in buy_signals:
        print(f"{signal['Name']:<15} {signal['Token']:<10} {signal['Current Price']:<15.2f} {signal['Support Level']:<15.2f}")
    print("--------------------------------------------------")

def get_stock_name(alice, token):
    instrument = alice.get_instrument_by_token('NSE', token)
    return instrument.name

# Initialize Aliceblue object and get credentials
alice = initialize_alice()

# List of tokens to analyze
tokens = [7,13,17903,21614,30108,22,25,15083,11703,1270,157,163,212,236,14418,263,21238,275,5900,16669,16675,317,335,341,2263,4668,
          371,383,404,422,10604,438,11373,2181,526,547,6994,10794,583,637,685,694,20374,11543,15141,4749,739,17094,5701,1901,772,8075,
          19943,10940,21690,14732,881,910,958,676,1023,4717,7406,13528,1174,10099,17875,11872,1232,10599,2303,9819,7229,1333,4244,1333,
          467,1348,1363,17939,1406,1394,4963,21770,18652,14366,11184,11184,220,11262,1512,1515,10726,11195,5258,29135,1594,1624,1633,
          13611,1660,6733,13270,11723,18096,1922,18564,11654,19234,1997,11483,24948,17818,10440,2031,13285,19061,4067,10999,31181,9581,
          2142,17534,4204,4503,2277,23650,6364,13751,14672,17963,15332,11630,20242,10738,2475,14413,2412,18365,11351,14299,2664,24184,
          10666,9590,14977,13147,2043,18391,15355,2885,2963,17971,21808,3045,3103,3150,3273,3351,13404,10243,3405,3721,3432,3456,3426,3499,
          11536,13538,3506,3518,10447,1964,8479,16713,11532,11287,3063,3718,3787,3812,7929
]

# Analyze each stock
buy_signals = []

for token in tokens:
    try:
        result = analyze_stock(alice, token)
        if result[2] == "BUY":
            buy_signals.append((result[0], result[1], result[3]))
    except Exception as e:
        print(f"Error analyzing token {token}: {e}")

# Adding stock names to the signals
for i in range(len(buy_signals)):
    buy_signals[i] = {
        'Token': buy_signals[i][0],
        'Current Price': buy_signals[i][1],
        'Support Level': buy_signals[i][2],
        'Name': get_stock_name(alice, buy_signals[i][0])
    }

# Print signals
print_signals(buy_signals)

Enter your user ID: 1141826
Enter your API key: uGmlgEfgvtLmlSfnPUvZJDuOfLOU6r0JVWlWpJezmeBBlGZuf63c6x1hBBmFdI7BcE15pQzpucaaNNJ4e8COP4LXqOSsSvLfMEh4qB6ZligOWjq5869N7hPO1TnvnoZ4
Error analyzing token 2031: If using all scalar values, you must pass an index
Error analyzing token 13285: If using all scalar values, you must pass an index
BUY Signals:
Name            Token      Current Price   Support Level  
-------------------------------------------------------
ABB-EQ          13         7670.50         7666.75        
ALKEM-EQ        11703      5682.75         5681.95        
AUBANK-EQ       21238      595.90          595.60         
BEL-EQ          383        312.85          309.30         
CUMMINSIND-EQ   1901       3545.75         3544.05        
DABUR-EQ        772        522.30          521.85         
DRREDDY-EQ      881        1214.85         1195.35        
HCLTECH-EQ      7229       1899.60         1891.70        
HINDPETRO-EQ    1406       387.75          386.20         
IOC-E